In [2]:
# installing dependencies 
%pip install --upgrade pip
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install pickle-mixin

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import pickle as pk
import os.path as path


     ---------------------------------------- 2.1/2.1 MB 9.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pickle-mixin: filename=pickle_mixin-1.0.2-py3-none-any.whl size=6008 sha256=d61d53a895e770ff03b2309e88e4a6b9aa4fc2aad54b4d1546c7b5fad14591c3
  Stored in directory: c:\users\hashi\appdata\local\pip\cache\wheels\3e\c6\e9\d1b0a34e1efc6c3ec9c086623972c6de6317faddb2af0a619c
Successfully built pickle-mixin
Note: you may need to restart the kernel to use updated packages.


In [3]:
# reading and formatting cifar-10 dataset

# utility function to read a batch file in the cifar-10-batches-py directory
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pk.load(fo, encoding='bytes')
    return dict

# utility function to gaussian norm numpy array to range [0, 1]
def norm(data: np.ndarray):
    norm = (data - np.mean(data)) / np.std(data)
    return (norm - np.min(norm)) / (np.max(norm) - np.min(norm))


# read and normalize train and test data
def read_train_test(dir: str='./cifar-10-batches-py', nrm: bool=True):
    TRAIN = []
    for i in range(5):
        batch = unpickle(path.join(dir, f'data_batch_{i+1}'))
        data, labels = batch[b'data'], np.reshape(np.array(batch[b'labels']), (10000, 1))
        if nrm: data = norm(data)
        TRAIN.append(np.hstack((data, labels)))

    # train split with shape 5, 100000, 3073 
    # 5 batches
    # 100000 images
    # 1024 red, 1024 blue, 1024 green, 1 label
    TRAIN = np.array(TRAIN)

    # reading test data same as abv
    test_batch = unpickle(path.join(dir, 'test_batch'))
    test_data, test_labels = test_batch[b'data'], np.reshape(np.array(test_batch[b'labels']), (10000, 1))
    if nrm: test_data = norm(test_data)
    TEST = np.hstack((test_data, test_labels))

    return TRAIN, TEST

TRAIN, TEST = read_train_test()

# example, batch 1 from train split
batch1_df = pd.DataFrame(TRAIN[1, :, :])
batch1_df


,0,1,2,3,4,5,6,7,8,9,...,3063,3064,3065,3066,3067,3068,3069,3070,3071,3072
0,0.137255,0.105882,0.098039,0.101961,0.094118,0.086275,0.090196,0.098039,0.098039,0.094118,...,0.686275,0.682353,0.682353,0.670588,0.670588,0.662745,0.662745,0.658824,0.658824,1.0
1,0.078431,0.078431,0.070588,0.074510,0.082353,0.090196,0.090196,0.090196,0.090196,0.090196,...,0.407843,0.466667,0.615686,0.639216,0.274510,0.258824,0.435294,0.380392,0.200000,6.0
2,0.454902,0.450980,0.607843,0.556863,0.388235,0.454902,0.552941,0.435294,0.431373,0.478431,...,0.070588,0.054902,0.031373,0.023529,0.019608,0.019608,0.070588,0.329412,0.486275,6.0
3,0.384314,0.356863,0.321569,0.270588,0.231373,0.235294,0.258824,0.400000,0.392157,0.364706,...,0.180392,0.341176,0.427451,0.443137,0.439216,0.470588,0.490196,0.517647,0.541176,8.0
4,0.333333,0.400000,0.427451,0.462745,0.192157,0.054902,0.129412,0.168627,0.117647,0.101961,...,0.482353,0.411765,0.345098,0.345098,0.333333,0.278431,0.250980,0.254902,0.258824,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.925490,0.913725,0.913725,0.913725,0.913725,0.913725,0.913725,0.913725,0.913725,0.913725,...,0.741176,0.745098,0.752941,0.752941,0.768627,0.768627,0.772549,0.776471,0.764706,8.0
9996,0.905882,0.905882,0.909804,0.901961,0.901961,0.905882,0.917647,0.925490,0.921569,0.921569,...,0.525490,0.564706,0.576471,0.576471,0.631373,0.674510,0.666667,0.666667,0.690196,8.0
9997,0.498039,0.545098,0.607843,0.584314,0.517647,0.619608,0.639216,0.619608,0.623529,0.556863,...,0.788235,0.772549,0.772549,0.784314,0.780392,0.780392,0.772549,0.752941,0.749020,7.0
9998,0.745098,0.784314,0.815686,0.815686,0.815686,0.815686,0.811765,0.800000,0.796078,0.796078,...,0.639216,0.631373,0.635294,0.635294,0.635294,0.635294,0.639216,0.713725,0.752941,2.0


In [55]:

# relu activation function
# relu = lambda x : x if x > 0 else 0
def relu(x):
    return x if x > 0 else 0

# softmax activation function
def softmax():
    pass

# vanilla neural network / mlp
class mlp:

    # default network structure
    # input data            x:  N x 1
    # input -> hidden1      m1: 64 x N   b1: 64 x 1
    # hidden1 -> hidden2    m2: 64 x 64  b2: 64 x 1
    # hidden2 -> output     m3: 10 x 64  b3: 10 x 1
    # y_h = softmax(m3*relu(m2*(relu(m1*x+b1))+b2)+b3)

    # notes
    # w weight matrix for the mlp
    # w is ragged since not each layer has the same dimension feature matrix
    # bias b1 b2 b3 are included in m1 m2 m3

    def __init__(self, hidden_activation=relu, output_activation=softmax, hidden_layers=2, hidden_dim=64, N=3072, output_dim=10):

        # initialize random weights for each layers feature matrix
        w = []
        m1= np.random.randn(hidden_dim, N)
        w.append(m1)

        for _ in range(hidden_layers-1):
            m = np.random.randn(hidden_dim, hidden_dim+1)
            w.append(m)

        mn = np.random.randn(output_dim, hidden_dim+1)
        w.append(mn)
        w = np.array(w)

        self.w = w
        self.hidden_activation = hidden_activation
        self.output_activation = output_activation
        self.input_dim = N
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        # finding number of parameters / weights
        self.num_params = sum(fm.shape[0]*fm.shape[1] for fm in w)
        

    def fit(self) -> None:
        pass

    # forward propogation
    # image data    x: 3072 x 1
    # prediction    y_h: 1 x 1
    def predict(self, x) -> int:
        y_h = x
        for i in range(len(self.w) - 1):
            y_h = relu(y_h @ self.w[i, :, :-1] + self.w[i, :, -1])
        
        y_h = softmax(y_h @ self.w[i+1, :, :-1] + self.w[i+1, :, -1])
        

    def __str__(self) -> str:
        res = ''

        res += f"{'input_dim:':<22}{str((self.input_dim, 1)):<15} {'activation: null':<15}\n"
        # feature matrix 1 summary
        # hidden layer summaries
        for i in range(len(self.w)-1):
            res += f"{'m'+ str(i+1) + '_dim:':<22}{str(self.w[i].shape):<15} activation: {str(self.hidden_activation.__name__):<15}\n"
        
        # output feature matrix summary
        res += f"{'m' + str(i+1) + '_dim:':<22}{str(self.w[i+1].shape):<15} activation: {str(self.output_activation.__name__):<15}\n"
        res += f"{'output_dim:':<22}{str((self.output_dim, 1)):<15} {'activation: null':<15}\n"

        # num of trainable parameters 
        res += f"{'num_parameters:':<22}{self.num_params}\n"
        return res



nn = mlp()
print(nn)

  

input_dim:            (3072, 1)       activation: null
m1_dim:               (64, 3072)      activation: relu           
m2_dim:               (64, 65)        activation: relu           
m2_dim:               (10, 65)        activation: softmax        
output_dim:           (10, 1)         activation: null
num_parameters:       201418



C:\Users\hashi\AppData\Local\Temp\ipykernel_21016\1458219953.py:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  w = np.array(w)
